In [30]:
#importing libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from openpyxl import load_workbook

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
url='https://www.prokabaddi.com'

In [15]:
def instant_browser():
    return webdriver.Chrome(r'C:\Users\divakar.kareddy\Hackathon\chromedriver_win32\chromedriver.exe')

In [34]:
def close_driver(driver):
    driver.close()

In [16]:
def invokebrowser(driver,url):
    '''
    instantiates chrome webdriver and browse given url and return webdriver 
    '''
    driver.get(url)
    time.sleep(1)
    driver.maximize_window()
    return driver

In [17]:
#Get the page data into soup object
def soup_website(driver):
    '''
    input-->webdriver instance
    output---> soup of the website invoked by driver'''
    soup=BeautifulSoup(driver.page_source)
    return soup

In [18]:
#Get the options available for player in the dropdown
def get_links(soup):
    '''
    input--->soup object of page
    output--->return the options present in the dropdown'''
    links=soup.find_all('div',{'class':'submenu-item'})[2].find_all('a')
    list_links=[url+links[i].attrs['href']+'/players' for i in range(len(links))]
    team_name=[links[i].attrs['href'].replace('/teams/','')\
                   [:-10].replace('-',' ').rstrip(' ').title() for i in range(len(links))]
    return (list_links,team_name)

In [70]:
def player_scrapper(tup,excel):
    links,teams=tup
    for i in range(len(links)):
        driver.get(links[i])
        time.sleep(2)
        page=BeautifulSoup(driver.page_source)
        playernames=page.find_all('div',{'class':'si-plyrName'})
        roles=page.find_all('div',{'class':'si-plyrRole'})
        playerprofiles=page.find_all('a',{'class':'si-plyrBox'})
        dit={}
        player_name=[(playernames[i].find_all('div',{'class':'si-txt1'})[0].text+""+\
                    playernames[i].find_all('div',{'class':'si-txt2'})[0].text).strip().title() for i in range(len(playernames))]
        role=[roles[i].text.strip() for i in range(len(roles))]
        player_profile=[url+playerprofiles[i].attrs['href'].strip() for i in range(len(playerprofiles))]
        dit['Player Name']=player_name
        dit['Role']=role
        dit['Profile Link']=player_profile
        team_df=pd.DataFrame(dit)
        if i==0:
            team_df.to_excel(excel,sheet_name=teams[i],engine = 'xlsxwriter',index=False)
        else:
            book = load_workbook(excel)
            writer = pd.ExcelWriter(excel, engine = 'openpyxl')
            writer.book = book
            team_df.to_excel(writer, sheet_name =teams[i],index=False)
            writer.save()
            writer.close()
        del team_df,dit,playernames,roles,role,playerprofiles,player_profile,player_name

In [71]:
driver=invokebrowser(instant_browser(),url)
try:
    player_scrapper(get_links(soup_website(driver)),"Teams.xlsx")
except Exception as e:
    print(e)
    print("error")
finally:
    close_driver(driver)